In [2]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=('PEPE',))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    # x.join()
    logging.info("Main    : all done")

22:33:35: Main    : before creating thread
22:33:35: Main    : before running thread
22:33:35: Thread PEPE: starting
22:33:35: Main    : wait for the thread to finish
22:33:35: Main    : all done


22:33:37: Thread PEPE: finishing


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/DataSet_Titanic.csv")

In [5]:
df

,Sobreviviente,Clase,Genero,Edad,HermEsp,PadHij
0,0,3,0,22.0,1,0
1,1,1,1,38.0,1,0
2,1,3,1,26.0,0,0
3,1,1,1,35.0,1,0
4,0,3,0,35.0,0,0
...,...,...,...,...,...,...
709,0,3,1,39.0,0,5
710,0,2,0,27.0,0,0
711,1,1,1,19.0,0,0
712,1,1,0,26.0,0,0


In [30]:
p = 'clase-0'

p.replace('clase-', '')

'0'

In [5]:
qualitative_variables_form = []
for column in df.drop(columns='Sobreviviente'):
    if len(set(df[column])) < 5:
        qualitative_variables_form.append({
            'name': column,
            'variables': list(set(df[column]))
        })
print(qualitative_variables_form)

[{'name': 'Clase', 'variables': [1, 2, 3]}, {'name': 'Genero', 'variables': [0, 1]}]


In [3]:
from sklearn.model_selection import train_test_split

data = df.drop(columns="Sobreviviente")
x_train, x_test, y_train, y_test = train_test_split(data,
                                                    df["Sobreviviente"], test_size=0.2,
                                                    random_state=123)


In [7]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(x_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [11]:
possible_targets = list(set(df.columns) - set(clf.feature_names_in_))

In [12]:
possible_targets

['Sobreviviente']

In [4]:
import joblib
model = joblib.dump(clf,"D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/titanic.joblib")

NameError: name 'clf' is not defined

In [ ]:
print(data.columns)

In [40]:
def get_rules(model, q_variables, q_variables_values, features, class_names, target):
    tree_ = model
    feature_name = [
        features[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    cont = 1

    def get_simplified_route(route):
        simplified_route = list(route)
        watched_classes_plus = []
        watched_classes_less = []
        done = False
        iteration = 0
        while (not done):
            analyzed_rule = {}
            analyzed_item = ""
            analyzed_sign = ""
            analyzed_value = []
            analyzed_type = ""
            for i in range(len(route)):
                if (iteration + i) < len(route):
                    actual = route[i + iteration]
                    if actual["item"] not in DT["q_variables"] and (analyzed_type == "number" or not analyzed_type):
                        if not analyzed_item:
                            analyzed_rule = actual
                            analyzed_type = "number"
                            analyzed_item = actual["item"]
                            analyzed_value = actual["value"]
                            if analyzed_item not in watched_classes_plus and actual["sign"] == " > ":
                                watched_classes_plus.append(analyzed_item)
                                analyzed_sign = " > "
                            elif analyzed_item not in watched_classes_less and actual["sign"] == " <= ":
                                watched_classes_less.append(analyzed_item)
                                analyzed_sign = " <= "
                        elif actual["item"] == analyzed_item and actual["sign"] == analyzed_sign:
                            try:
                                if analyzed_sign == " > ":
                                    if actual["value"] > analyzed_value:
                                        simplified_route.remove(analyzed_rule)
                                        analyzed_value = actual["value"]
                                        analyzed_rule = actual
                                    elif actual["value"] <= analyzed_value:
                                        simplified_route.remove(actual)
                                elif analyzed_sign == " <= ":
                                    if actual["value"] <= analyzed_value:
                                        simplified_route.remove(analyzed_rule)
                                        analyzed_value = actual["value"]
                                        analyzed_rule = actual
                                    elif actual["value"] > analyzed_value:
                                        simplified_route.remove(actual)
                            except Exception as e:
                                str(e)
                                break
                    elif analyzed_type == "object" or not analyzed_type:
                        if not analyzed_item:
                            analyzed_rule = actual
                            analyzed_type = "object"
                            analyzed_sign = "es :"
                            analyzed_item = actual["item"]
                            analyzed_value = actual["value"]
                        elif actual["item"] == analyzed_item:
                            try:
                                if all(val in analyzed_value for val in actual["value"]):
                                    simplified_route.remove(analyzed_rule)
                                    analyzed_value = actual["value"]
                                elif all(analyzed_val in actual["value"] for analyzed_val in analyzed_value):
                                    simplified_route.remove(actual)
                            except Exception as e:
                                str(e)
                                break
                else:
                    break
            iteration += 1
            if iteration >= len(route):
                done = True
        return simplified_route

    def get_rule(name, threshold, bigger):
        rule = [{
            "item": name,
            "sign": "",
            "value": []
        }]
        if name in q_variables:
            new_args = []
            first = True
            # condition = "es : "
            rule[0]["sign"] = "es: "
            for variables in q_variables_values:
                if variables["column"] == name:
                    for values in variables["values"]:
                        if (not bigger and values["previous_value"] <= threshold) or (
                                bigger and values["previous_value"] > threshold):
                            new_args.append(values["new_value"])
            rule[0]["value"] = new_args
        else:
            if not bigger:
                rule[0]["sign"] = " <= "
            else:
                rule[0]["sign"] = " > "
            rule[0]["value"] = [f"{np.round(threshold, 3)}"]

        return rule

    def get_text_route(route):
        text_route = []
        for val in route:
            if val["item"] not in q_variables:
                text_route += [(f"({val['item']} {val['sign']} {val['value'][0]})")]
            else:
                values = ""
                first = True

                for i in range(len(val['value'])):
                    if first:
                        values += f"{val['value'][i]}"
                        first = False
                    else:
                        values += f" o {val['value'][i]}"
                text_route += [(f"{val['item']} {val['sign']} {values}")]
        return text_route

    def recurse(node, route, routes):

        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            left_path_rules, right_path_rules = list(route), list(route)
            left_path_rules += get_rule(name, threshold, bigger=False)
            recurse(tree_.children_left[node], left_path_rules, routes)
            right_path_rules += get_rule(name, threshold, bigger=True)
            recurse(tree_.children_right[node], right_path_rules, routes)
        else:
            text_route_original = get_text_route(route)
            text_route_original += [(tree_.value[node], tree_.n_node_samples[node])]
            simplified_route = get_simplified_route(route)
            text_route = get_text_route(simplified_route)
            text_route += [(tree_.value[node], tree_.n_node_samples[node])]
            routes += [text_route]

    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]

    rules = []
    for path in paths:
        rule = "Si "

        for p in path[:-1]:
            if rule != "Si ":
                rule += " y "
            rule += str(p)
        rule += " entonces "
        if class_names is None:
            rule += "respuesta: " + str(np.round(path[-1][0][0][0], 3))
        else:
            classes = path[-1][0][0]
            position = np.argmax(classes)
            rule += f"{target}: {class_names[position]} (con probabilidad: " \
                    f"{np.round(100.0 * classes[position] / np.sum(classes), 2)}%)"
        rule += f" | basado en {path[-1][1]:,} muestras"
        rules += [f"Regla {cont}: {rule}"]
        cont += 1

    return rules

In [ ]:
rules = get_rules(DT, DT["model"].tree_, DT["features"], DT["class_names"], DT["target"])
for r in rules:
    print(r)
    print()

In [ ]:
from dtreeviz import model


def graph_tree(DT):
    viz = model(
        DT["model"],
        X_train=x_train,
        y_train=y_train,
        feature_names=list(DT["features"]),
        class_names=DT["class_names"]
    )
    try:
        svg = open(viz.view().save_svg(), "rb").read()
        encoded = base64.b64encode(svg)
        svg_encoded = "data:image/svg+xml;base64,{}".format(encoded.decode())
        return svg_encoded, viz
    except Exception as e:
        print(e)
        return "", viz

In [ ]:
x, viz = graph_tree(DT)

In [ ]:
viz.view().show()

In [ ]:
print(list(DT["features"]))

In [10]:
from model.model import ClassificationTrainedModel

In [8]:
import joblib
from sklearn.ensemble import RandomForestClassifier

random_forest_model: RandomForestClassifier = joblib.load(
    "D:/CARRERA/TESIS/forest_explainer_tesis/datasets/Titanic/titanic.joblib")

q_variables_values_list = [
    {
        "column_name": "Genero",
        "variables": [
            {
                "old_value": 0,
                "new_value": "Masculino"
            },
            {
                "old_value": 1,
                "new_value": "Femenino"
            },
        ]
    },
    {
        "column_name": "Clase",
        "variables": [
            {
                "old_value": 1,
                "new_value": "Primera Clase"
            },
            {
                "old_value": 2,
                "new_value": "Segunda Clase"
            },
            {
                "old_value": 3,
                "new_value": "Tercera Clase"
            },
        ]
    }
]

target_description = [
    {
        "column_name": "Sobreviviente",
        "variables": [
            {
                "old_value": 0,
                "new_value": "Muere"
            },
            {
                "old_value": 1,
                "new_value": "Vive"
            },
        ]
    }
]

In [11]:
explainer_classifier = ClassificationTrainedModel(name="Titanic", df=df, predictors_description="features_description",
                                                  target="Sobreviviente", test_size=0.2, random_state=123,
                                                  model=random_forest_model, model_description="",
                                                  target_description=target_description[0],
                                                  q_variables_values_list=q_variables_values_list)


{'column_name': 'Genero', 'variables': [{'old_value': 0, 'new_value': 'Masculino'}, {'old_value': 1, 'new_value': 'Femenino'}]}
{'column_name': 'Clase', 'variables': [{'old_value': 1, 'new_value': 'Primera Clase'}, {'old_value': 2, 'new_value': 'Segunda Clase'}, {'old_value': 3, 'new_value': 'Tercera Clase'}]}
Creating data_table...


KeyError: 'column'

In [57]:
from sklearn import tree
from sklearn.tree import _tree
import base64

DT = {
    # "model": tree.DecisionTreeClassifier(max_depth=2).fit(x_train, y_train),
    "model": random_forest_model.estimators_[25],
    "features": data.columns,
    "class_names": ["Muere", "Vive"],
    "target": "Sobreviviente",
    "q_variables": ["Genero", "Clase"],
    "q_variables_values": q_variables_values_list,
}

In [58]:
# rules = get_rules(DT, DT["model"].tree_, DT["features"], DT["class_names"], DT["target"])
rules = get_rules(model=DT["model"].tree_, q_variables=DT["q_variables"], q_variables_values=DT["q_variables_values"], features=DT["features"], class_names=DT["class_names"], target=DT["target"])
for r in rules:
    print(r)
print()

Regla 1: Si (PadHij  <=  0.5) y Genero es:  Masculino entonces Sobreviviente: Muere (con probabilidad: 83.16%) | basado en 188 muestras
Regla 2: Si (PadHij  <=  0.5) y Genero es:  Femenino entonces Sobreviviente: Vive (con probabilidad: 72.88%) | basado en 78 muestras
Regla 3: Si (PadHij  >  0.5) y Genero es:  Masculino entonces Sobreviviente: Muere (con probabilidad: 66.67%) | basado en 47 muestras
Regla 4: Si (PadHij  >  0.5) y Genero es:  Femenino entonces Sobreviviente: Vive (con probabilidad: 81.94%) | basado en 46 muestras

